In [1]:
import pandas as pd

In [2]:
df= pd.read_csv(r'C:\Users\anipr\Downloads\PII metadata - Sheet1.csv')

In [3]:
import pandas as pd
import re

def parse_timestamp(ts):
    """
    Converts a timestamp string to seconds.
    Handles formats like '13.269', '1.26.523', '4.05.939', etc.
    """
    ts = ts.strip().replace(' ', '')
    # Fix common typos like '49-223' instead of '49.223'
    if '-' in ts and ts.count('-') == 1 and ts.replace('-', '').replace('.', '').isdigit():
        ts = ts.replace('-', '.')
    parts = ts.split('.')
    if len(parts) == 2:
        # Format: seconds.milliseconds
        try:
            return float(parts[0]) + float('0.' + parts[1])
        except:
            return None
    elif len(parts) == 3:
        # Format: min.sec.ms
        try:
            minutes = int(parts[0])
            seconds = int(parts[1])
            milliseconds = int(parts[2])
            return minutes * 60 + seconds + milliseconds / 1000
        except:
            return None
    else:
        try:
            return float(ts)
        except:
            return None

def parse_range(rng):
    """
    Parses a range string like '1.26.523-1.27.342' or '13.269-14.367'
    Returns (start_seconds, end_seconds)
    """
    rng = rng.strip().replace(' ', '')
    if not rng or rng.count('-') != 1:
        return None
    start, end = rng.split('-')
    start_sec = parse_timestamp(start)
    end_sec = parse_timestamp(end)
    if start_sec is not None and end_sec is not None:
        return (start_sec, end_sec)
    return None

# Load the CSV file
file_path = r'C:\Users\anipr\Downloads\PII metadata - Sheet1.csv'
df = pd.read_csv(file_path)

# Get the data row (with a valid 'File Name')
first_row = df[df['File Name'].notna()].iloc[3]

# Extract all timestamp columns except 'Sl #' and 'File Name'
timestamp_columns = [col for col in first_row.index if col not in ['Sl #', 'File Name']]

# Parse and convert all ranges
ranges_in_seconds = []
for col in timestamp_columns:
    val = str(first_row[col])
    parsed = parse_range(val)
    if parsed:
        ranges_in_seconds.append(parsed)

print(ranges_in_seconds)

[(10.153, 10.778), (13.356, 13.781), (14.159, 15.404), (17.687, 18.347), (45.686, 46.411), (74.327, 75.047), (83.772, 84.374)]


In [4]:
len(ranges_in_seconds)

7

In [5]:
df.head()

,Sl #,File Name,PII-name,PII-name.1,PII-name.2,PII-name.3,PII-name.4,PII-Company name,PII-name.5,PII-Name,PII- Name,PII-Name.1,PII-Name.2,PII-Name.3,PII-Name.4
0,1.0,out-70001-919417328881-20250530-160219-1748601...,13.269- 14.367,17.371-18.521,17.430 - 18.354,22.239-25.564,19.229 - 20.116,26.279-28.496,45.846-46.830,48.529-49-223,58.328-59.146,1.26.523-1.27.342,1.49.540-1.50.400,4.05.939-4.06.622,4.16.339-4.16.846
1,NaN,NaN,PII-Name,PII-name,PII-Company Name,PII-name,PII-name,PII-Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,out-8968336334-1001-20250530-150555-1748597755...,05.855-06.423,08.916-09.395,09.652-11.063,11.808-12.358,35.219-35.752,1.27.321-1.28.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,PII-Name,PII-name,PII-Name,PII-name,PII-name,PII-Name,PII-Name,PII-Name,PII-Name,NaN,NaN,NaN,NaN
4,3.0,out-9463065622-1001-20250530-152345-1748598825...,03.760-04.173,05.953-06.592,23.209-23.891,33.260-34.049,48.065-48.650,53.225-53.813,01.30.170-01.30.704,01.48.369-01.49.173,02.02.844-02.03.375,NaN,NaN,NaN,NaN


In [6]:
ranges_in_seconds

[(10.153, 10.778),
 (13.356, 13.781),
 (14.159, 15.404),
 (17.687, 18.347),
 (45.686, 46.411),
 (74.327, 75.047),
 (83.772, 84.374)]

In [7]:
# Sort the list by the start time of each range
sorted_ranges = sorted(ranges_in_seconds, key=lambda x: x)

print(sorted_ranges)

[(10.153, 10.778), (13.356, 13.781), (14.159, 15.404), (17.687, 18.347), (45.686, 46.411), (74.327, 75.047), (83.772, 84.374)]


In [8]:
ground_truth = sorted_ranges

# Your model predictions
predictions = [(10.92, 11.16), (13.03, 13.55), (13.75, 14.15), (45.87, 46.83), (74.63, 75.03), (83.99, 84.43)]



In [9]:
def calculate_overlap(range1, range2):
    """
    Calculate the overlap duration between two timestamp ranges.

    Parameters:
        range1 (tuple): (start1, end1) in seconds
        range2 (tuple): (start2, end2) in seconds

    Returns:
        float: Overlap duration in seconds. 0 if no overlap.
    """
    start1, end1 = range1
    start2, end2 = range2

    # Find the latest start time and earliest end time
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)

    # If the intervals overlap, overlap_start < overlap_end
    if overlap_start < overlap_end:
        return overlap_end - overlap_start
    else:
        return 0


def evaluate_ranges_any_overlap(ground_truth, predictions):
    """
    Evaluate predictions against ground truth where ANY overlap counts as TP
    Returns: (true_positives, false_positives, false_negatives)
    """
    tp, fp, fn = 0, 0, 0
    matched_gt = set()
    matched_pred = set()
    
    # Check all predictions
    for pred_idx, pred_range in enumerate(predictions):
        found_overlap = False
        
        # Check against all ground truth ranges
        for gt_idx, gt_range in enumerate(ground_truth):
            if calculate_overlap(pred_range, gt_range) > 0:
                tp += 1
                matched_gt.add(gt_idx)
                matched_pred.add(pred_idx)
                found_overlap = True
                break  # Stop after first match
        
        if not found_overlap:
            fp += 1  # No overlap with any ground truth
    
    # False negatives = ground truths with no matching predictions
    fn = len(ground_truth) - len(matched_gt)
    
    return tp, fp, fn

In [10]:
tp, fp, fn = evaluate_ranges_any_overlap(ground_truth, predictions)

# Calculate standard metrics
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
f2_score = 5 * (precision * recall) / (4 * precision + recall) if (4 * precision + recall) > 0 else 0

print(f"True Positives: {tp}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1_score:.3f}")
print(f"F2 Score: {f2_score:.3f}")

True Positives: 5
False Positives: 1
False Negatives: 3
Precision: 0.833
Recall: 0.625
F1 Score: 0.714
F2 Score: 0.658
